<a href="https://colab.research.google.com/github/kliang696/Udemy_Analysis/blob/main/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
from scipy.stats import mstats
from scipy.stats.mstats import winsorize

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


In [30]:
my_df = pd.read_csv("/content/drive/MyDrive/udemy/Course_info.csv")

## Exclude unnecessary data

In [31]:
df = my_df.drop(columns=['course_url', 'instructor_name','instructor_url','headline', 'title'])

In [32]:
df = df[df["num_subscribers"]>0]

In [33]:
df = df[df["is_paid"]==True]

In [34]:
df = df[df['content_length_min']>0]

In [35]:
df = df[df['num_lectures']>0]

In [36]:
print(df.isna().sum())

id                      0
is_paid                 0
price                   0
num_subscribers         0
avg_rating              0
num_reviews             0
num_comments            0
num_lectures            0
content_length_min      0
published_time          0
last_update_date      119
category                0
subcategory             0
topic                 651
language                0
dtype: int64


In [37]:
df.dropna(inplace=True)

## Feature Engineer

In [38]:
df['published_time'] = pd.to_datetime(df['published_time'])
df['published_time'] = df['published_time'].dt.strftime('%Y-%m-%d')

df['last_update_date'] = pd.to_datetime(df['last_update_date'])
df['last_update_date'] = df['last_update_date'].dt.strftime('%Y-%m-%d')

In [39]:
df['total_days_maintained']=(pd.to_datetime(df['last_update_date']) - pd.to_datetime(df['published_time'])).dt.days

In [40]:
df.loc[df["total_days_maintained"] <= 0, "total_days_maintained"] = 0

In [41]:
df['total_days_maintained'].describe()

count    169360.000000
mean        299.699598
std         520.415914
min           0.000000
25%           0.000000
50%          15.000000
75%         401.000000
max        4098.000000
Name: total_days_maintained, dtype: float64

In [42]:
date_string = "2022 10 30"
date_object = pd.to_datetime(date_string, format='%Y %m %d')
df['course_age']=(date_object - pd.to_datetime(df['published_time'])).dt.days

In [43]:
df['course_age'].describe()

count    169360.000000
mean       1050.095288
std         766.526419
min          25.000000
25%         473.000000
50%         849.000000
75%        1493.000000
max        4582.000000
Name: course_age, dtype: float64

In [44]:
df["total_amount"] = df["num_subscribers"] * df["price"]

## Outlier and data transformation

In [45]:
num=["price","num_subscribers","avg_rating","num_reviews","num_comments","num_lectures","content_length_min","course_age","total_days_maintained","total_amount"]

In [46]:
lower_threshold = 0
upper_threshold = 0.1

for i in range(len(num)):
    column = num[i]
    df[column] = winsorize(df[column], limits=(lower_threshold, upper_threshold))

In [71]:
df.describe()

,id,price,num_subscribers,avg_rating,num_reviews,num_comments,num_lectures,content_length_min,total_days_maintained,course_age,total_amount
count,1.693600e+05,169360.000000,169360.000000,169360.000000,169360.000000,169360.000000,169360.000000,169360.000000,169360.000000,169360.000000,169360.000000
mean,2.910096e+06,79.018398,1092.747243,3.929586,67.107056,15.935427,34.278342,240.882026,242.294479,999.252391,82609.072724
std,1.341340e+06,68.340494,1735.777913,1.322655,98.583913,22.079179,25.481093,197.584754,356.446170,654.737450,141837.355711
min,2.762000e+03,0.100000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,25.000000,0.100000
25%,1.818672e+06,19.990000,30.000000,3.944444,4.000000,1.000000,14.000000,87.000000,0.000000,473.000000,1199.520000
50%,3.122529e+06,49.990000,186.000000,4.357143,19.000000,5.000000,26.000000,166.000000,15.000000,849.000000,10074.960000
75%,4.082374e+06,124.990000,1219.000000,4.650000,77.000000,20.000000,48.000000,338.000000,401.000000,1493.000000,77842.215000
max,4.913954e+06,199.990000,5362.000000,4.890625,309.000000,69.000000,87.000000,661.000000,1025.000000,2203.000000,440377.980000


In [72]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer


scaler = StandardScaler()

 #Fit the scaler to the data set
scaler.fit(df[num])

  # Apply the normalization to the data set
normalized_data = scaler.transform(df[num])


df_normalized_num = pd.DataFrame(normalized_data, columns=df[num].columns)


In [74]:
df_normalized_num.describe()

,price,num_subscribers,avg_rating,num_reviews,num_comments,num_lectures,content_length_min,course_age,total_days_maintained,total_amount
count,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05
mean,9.867717e-17,-5.638695e-17,9.827440e-16,4.296149e-17,-3.222112e-17,2.416584e-17,3.289239e-17,-6.981242e-17,8.055279e-18,4.564658e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-1.154786e+00,-6.289690e-01,-2.970992e+00,-6.807120e-01,-7.217422e-01,-1.306005e+00,-1.214075e+00,-1.488009e+00,-6.797526e-01,-5.824221e-01
25%,-8.637423e-01,-6.122617e-01,1.123382e-02,-6.401373e-01,-6.764505e-01,-7.958215e-01,-7.788176e-01,-8.037633e-01,-6.797526e-01,-5.739658e-01
50%,-4.247626e-01,-5.223882e-01,3.232576e-01,-4.879822e-01,-4.952838e-01,-3.248827e-01,-3.789880e-01,-2.294856e-01,-6.376703e-01,-5.113909e-01
75%,6.726866e-01,7.273576e-02,5.446743e-01,1.003508e-01,1.840913e-01,5.385051e-01,4.915271e-01,7.541176e-01,4.452453e-01,-3.360801e-02
max,1.770136e+00,2.459569e+00,7.266005e-01,2.453683e+00,2.403383e+00,2.069056e+00,2.126273e+00,1.838525e+00,2.195865e+00,2.522396e+00


In [75]:
print(df_normalized_num.isna().sum())

price                    0
num_subscribers          0
avg_rating               0
num_reviews              0
num_comments             0
num_lectures             0
content_length_min       0
course_age               0
total_days_maintained    0
total_amount             0
dtype: int64


In [49]:
df_normalized_num.shape

(169360, 10)

In [50]:
#df_normalized_num.info()

In [51]:
df.shape

(169360, 18)

In [52]:
#df.info()

In [54]:
df = df.reset_index(drop=True)
df_normalized_num = df_normalized_num.reset_index(drop=True)


In [55]:
df_normalized_num["category"]=df["category"]

In [57]:
df_normalized_num["topic"]=df["topic"]
df_normalized_num["subcategory"]=df["subcategory"]
df_normalized_num["language"]=df["language"]

In [76]:
print(df_normalized_num.isna().sum())

price                    0
num_subscribers          0
avg_rating               0
num_reviews              0
num_comments             0
num_lectures             0
content_length_min       0
course_age               0
total_days_maintained    0
total_amount             0
dtype: int64


In [59]:
df_normalized_num.describe()

,price,num_subscribers,avg_rating,num_reviews,num_comments,num_lectures,content_length_min,course_age,total_days_maintained,total_amount
count,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05,1.693600e+05
mean,9.867717e-17,-5.638695e-17,9.827440e-16,4.296149e-17,-3.222112e-17,2.416584e-17,3.289239e-17,-6.981242e-17,8.055279e-18,4.564658e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-1.154786e+00,-6.289690e-01,-2.970992e+00,-6.807120e-01,-7.217422e-01,-1.306005e+00,-1.214075e+00,-1.488009e+00,-6.797526e-01,-5.824221e-01
25%,-8.637423e-01,-6.122617e-01,1.123382e-02,-6.401373e-01,-6.764505e-01,-7.958215e-01,-7.788176e-01,-8.037633e-01,-6.797526e-01,-5.739658e-01
50%,-4.247626e-01,-5.223882e-01,3.232576e-01,-4.879822e-01,-4.952838e-01,-3.248827e-01,-3.789880e-01,-2.294856e-01,-6.376703e-01,-5.113909e-01
75%,6.726866e-01,7.273576e-02,5.446743e-01,1.003508e-01,1.840913e-01,5.385051e-01,4.915271e-01,7.541176e-01,4.452453e-01,-3.360801e-02
max,1.770136e+00,2.459569e+00,7.266005e-01,2.453683e+00,2.403383e+00,2.069056e+00,2.126273e+00,1.838525e+00,2.195865e+00,2.522396e+00


## modelling

In [105]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.9 MB/s eta 0:00:00


In [61]:
from catboost import CatBoostRegressor

In [62]:
X = df_normalized_num.drop(columns=['price'])
y = df_normalized_num['price']

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [64]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [65]:
X.dtypes 

num_subscribers          float64
avg_rating               float64
num_reviews              float64
num_comments             float64
num_lectures             float64
content_length_min       float64
course_age               float64
total_days_maintained    float64
total_amount             float64
category                  object
topic                     object
subcategory               object
language                  object
dtype: object

In [66]:
categoricalcolumns = X.select_dtypes(include=["object"]).columns.tolist()
print("Names of categorical columns : ", categoricalcolumns)
#Get location of categorical columns
cat_features = [X.columns.get_loc(col) for col in categoricalcolumns]
print("Location of categorical columns : ",cat_features)

Names of categorical columns :  ['category', 'topic', 'subcategory', 'language']
Location of categorical columns :  [9, 10, 11, 12]


In [67]:
from catboost import Pool
#Creating pool object for train dataset. we give information of categorical fetures to parameter cat_fetaures
train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features
                 )
#Creating pool object for test dataset
test_data = Pool(data=X_test,
                  label=y_test,
                  cat_features=cat_features
                 )

In [68]:
from catboost import CatBoostRegressor

In [69]:
model = CatBoostRegressor(depth=8, learning_rate=0.01, l2_leaf_reg=5,loss_function="RMSE")

In [70]:
model.fit(X_train, y_train,verbose=False,eval_set=(X_test, y_test))

CatBoostError: ignored